In [ ]:
!pip install undetected-chromedriver

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from datetime import datetime
import random
import re
import time
import undetected_chromedriver as uc
from selenium import webdriver


In [ ]:
def get_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36')
  options.add_argument("--incognito")
  driver = uc.Chrome(options = options)
  driver.set_page_load_timeout(60)
  driver.set_window_size(1366, 768)
  driver.set_window_position(0, 0)
  driver.delete_all_cookies()
  driver.execute_script("localStorage.clear();")
  driver.execute_script("sessionStorage.clear();")
  driver.execute_script("history.go(0);")
  driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
    'source': 'Object.defineProperty(navigator, "webdriver", {get: () => undefined})'
  })
  return driver


def get_links(driver, url):
    actions = ActionChains(driver)
    driver.get(url)
    time.sleep(random.uniform(0.5, 1.5))
    try:
        button = driver.find_element(by=By.XPATH, value='/html/body/div/div/div[2]/div/div[2]/div/div/div/footer/div/button[2]')
        actions.move_to_element(button).perform()
        button.click()
        time.sleep(random.uniform(0.8, 1.2))
    except:
        pass
    increment = 100
    position = 0
    while True:
        driver.execute_script("window.scrollTo(0, " + str(increment) + ");")
        
        elements = driver.find_elements(by=By.XPATH, value='//*[@id="root"]/div/div/div/main/div/section/div/div/div/div/div/a')
        time.sleep(random.uniform(0.2, 0.8))
        links = [element.get_attribute('href') for element in elements]
        if position == 0:
            try:
                actions.move_to_element(elements[0]).perform()
            except:
                pass
            position += 1
        else:
            try:
                actions.move_by_offset(random.uniform(-20, 20), random.uniform(-100, 200))
            except:
                pass
        increment += 300
        if len(elements) >= 30 or increment >= 10000:
            break
    links = [element.get_attribute('href') for element in elements]
    display(len(links))
    display(links)
    return links

In [ ]:
links = []
last_size = 0
pg = 1
driver = get_driver()
while True:
    url = r'https://www.coches.net/segunda-mano/?MakeId=27&MinPrice=4000&MinYear=2012&MaxYear=2022&MinKms=10000&MaxKms=180000&pg=' + str(pg)
    print('url:',url)
    links.extend(get_links(driver, url))
    print('links len:',len(links))
    if last_size == len(links) or len(links) >= 1000:
        break
    last_size = len(links)
    pg += 1


In [ ]:
links

In [ ]:
import pandas as pd
links
df = pd.DataFrame(links)
df.to_csv("coches_link.csv")

In [ ]:
import pandas as pd
urls_coches = pd.read_csv("coches_link.csv")

# Crear una lista vacía para almacenar los enlaces leídos
urls_leidos = []


In [ ]:
for c in urls_coches["url"]:
  print(c.replace("#",""))
  break

In [ ]:
urls_coches

In [ ]:
coches_df = coches_df.drop(['Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1',
       'Unnamed: 0'], axis=1)
coches_df

In [ ]:
coches_df["url"]

In [2]:
import numpy as np
import pandas as pd
coches_df = pd.read_csv("coches_data.csv")
for data in coches_df.columns:
  try:
    if data.startswith('Unnamed'):
      coches_df = coches_df.drop([data], axis=1)
  except:
    pass
coches_df.to_csv("bmw_mazda.csv")

In [ ]:
from datetime import datetime
import numpy as np
coches_df = pd.read_csv("coches_data.csv")
coches_df = coches_df.drop_duplicates(subset='url')
for data in coches_df.columns:
  try:
    if data.startswith('Unnamed'):
      coches_df = coches_df.drop([data], axis=1)
  except:
    pass
coches_df.to_csv("coches_data.csv")
driver = get_driver()
actions = ActionChains(driver)
coches_df["url"] = coches_df["url"].astype("str")

# Realizar replace("#","") en los valores de urls_coches["url"]
urls_coches["url"] = urls_coches["url"].apply(lambda x: x.replace("#", ""))

# Identificar las filas de urls_coches["url"] que están presentes en coches_df["url"]
rows_to_drop = urls_coches[urls_coches["url"].isin(coches_df["url"])]

# Contar cuántas filas se van a eliminar
num_rows_to_drop = len(rows_to_drop)

# Eliminar las filas de urls_coches["url"] que coinciden con coches_df["url"]
urls_coches.drop(rows_to_drop.index, inplace=True)

# Mostrar cuántas filas se eliminaron
print(f"Se eliminaron {num_rows_to_drop} filas.")
for c in urls_coches["url"]:
  # Si el enlace ya ha sido leído, saltar al siguiente enlace
  if c.replace("#","") in coches_df["url"]:
    print("Continue")
    continue

  driver.get(c.replace("#",""))
  try:
    actions.move_by_offset(random.uniform(-20, 20), random.uniform(-100, 200))
  except:
    pass
  time.sleep(random.uniform(0.5, 1.5))
  try:
    button = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div/div[2]/div/div[3]/div/div/div/footer/div/button[2]')
    try:
      actions.move_to_element(button).perform()
    except:
      pass
    button.click()
    time.sleep(random.uniform(0.8, 1.2))
  except:
    pass

  json_text = driver.find_element(by=By.XPATH, value='/html/body/script[3]')
  json_text_content = json_text.get_attribute("innerHTML")
  print('content:', json_text_content)
  import re
  import json
  import pandas as pd

  # extraer la cadena JSON del texto plano
  match = re.search(r'JSON.parse\("(.*?)"\);', json_text_content)

  json_string = match.group(1)
  json_string = re.sub(r'\\([\\"])', r'\1', json_string) # eliminar las barras invertidas escapadas
  json_string = json_string.replace('\r', '') # eliminar los caracteres "\r"

  print(json_string)
  # analizar la cadena JSON
  json_data = json.loads(json_string)
  
  # crear un DataFrame a partir de los datos analizados
  df = pd.json_normalize(json_data)
  columnas = ['ad.fuelType', 'ad.fuelTypeId', 'ad.color', 'ad.id', 'ad.km', 'ad.make', 'ad.makeId', 'ad.model', 'ad.modelId', 'ad.province', 'ad.provinceId', 'ad.price', 'ad.title', 'ad.transmissionType', 'ad.transmissionTypeId', 'ad.year', 'pageTrackingData.data.seller_type', 'pageTrackingData.data.seller_type_id', 'pageTrackingData.data.body_types', 'pageTrackingData.data.body_types_id', 'vehicleInfo.specifications.cubicCapacity', 'vehicleInfo.specifications.doors', 'vehicleInfo.specifications.hp']
  df = df.reindex(columns=columnas, fill_value=np.nan)
  df = df.rename(columns=lambda x: x.split('.')[-1])
  df["url"] = c.replace("#","")
  df["date"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  coches_df = pd.concat([coches_df, df])
  for data in coches_df.columns:
    try:
      if data.startswith('Unnamed'):
        coches_df = coches_df.drop([data], axis=1)
    except:
      pass
  coches_df = coches_df.drop_duplicates(subset='url')
  coches_df.to_csv("coches_data.csv")


In [ ]:
coches_df

In [ ]:
ad.fuelType
ad.fuelType
ad.fuelTypeId
ad.color
ad.id
ad.km
ad.make
ad.makeId
ad.model
ad.modelId
ad.province
ad.provinceId
ad.price
ad.title
ad.transmissionType
ad.transmissionTypeId
ad.year
pageTrackingData.data.seller_type
pageTrackingData.data.seller_type_id
pageTrackingData.data.body_types
pageTrackingData.data.body_types_id
vehicleInfo.specifications.cubicCapacity
vehicleInfo.specifications.doors
vehicleInfo.specifications.hp

In [ ]:
for x in df.columns:
    print(x)